In [24]:
import numpy as np
from PIL import Image
from google.colab import files

In [25]:
 # funtion to convert text to Binary String

def text_to_binary(text):
  return ''.join(format(ord(char), '08b') for char in text)



# function to convert a Binary string back to text

def binary_to_text(binary_string):
  """Ensures a binary string is a multiple of 8"""
  if len(binary_string) % 8 != 0:
    binary_string = binary_string + '0' * (8 - len(binary_string) % 8)

  chars = []
  for i in range(0, len(binary_string), 8):
    byte = binary_string[i:i+8]
    chars.append(chr(int(byte,2)))
  return ''.join(chars)



# Hiding Text in the LSB of an image

def hide_text(image_path, secret_text, output_path):

  try:
    img = Image.open(image_path).convert("RGB")
    pixels = np.array(img)

    # Convert secret text to binary, and add a delimiter to mark the end
    binary_secret_text = text_to_binary(secret_text) + '1111111111111110' # A common delimiter (e.g., '###' in binary)


    # Calculate maximum embeddable bits
    max_bits = pixels.shape[0] * pixels.shape[1] * 3 # R, G, B channels
    if len(binary_secret_text) > max_bits:
      print(f"Error: Secret message is too long for the image. Max capacity: {max_bits} bits.")
      return


    data_idx = 0
    for r in range(pixels.shape[0]):
            for c in range(pixels.shape[1]):
                for channel in range(3): # Iterate R, G, B channels
                    if data_idx < len(binary_secret_text):
                        current_pixel_value = pixels[r, c, channel]
                        # Clear the LSB and set it to the secret bit
                        pixels[r, c, channel] = (current_pixel_value & 0b11111110) | int(binary_secret_text[data_idx])
                        data_idx += 1
                    else:
                        break # All data embedded
                if data_idx >= len(binary_secret_text):
                    break
            if data_idx >= len(binary_secret_text):
                break

    stego_img = Image.fromarray(pixels)
    stego_img.save(output_path)
    print(f"Text hidden successfully in '{output_path}' within Colab's session")

    # files.download(output_path) # Triggers download to user's local machine
    # print(f"'{output_path}' has been prompted for download to your local machine.")

  except Exception as e:
    print(f"An error occurred: {e}")





# Extracting text from the LSB of an image

def extract_text(stego_img_path):
  try:
    img = Image.open(stego_img_path).convert("RGB")
    pixels = np.array(img)

    binary_extracted_text = ""
    delimiter = '1111111111111110'
    delimiter_len = len(delimiter)

    for r in range(pixels.shape[0]):
            for c in range(pixels.shape[1]):
                for channel in range(3): # Iterate R, G, B channels
                    lsb = pixels[r, c, channel] & 0b00000001
                    binary_extracted_text += str(lsb)

                    if binary_extracted_text.endswith(delimiter):
                        return binary_to_text(binary_extracted_text[:-delimiter_len])
    return "Delimeter not found or message incomplete."

  except Exception as e:
    print(f"An error occured: {e}")
    return None

In [26]:

# Interactive Choice


while True:
  print("\n\n--- Steganography Tool ---")
  print("1. Hide text into an image")
  print("2. Extract text from an image")
  print("3. Exit")

  choice = input("\n\nEnter your choice (1,2 or 3): ")

  if choice == '1':
    print("\n--- Hide Text ---")
    print("Please upload your cover image:")
    uploaded = files.upload()

    if not uploaded:
      print("No file uploaded, Returning to the main menu. ")
      continue

    cover_image_file = list(uploaded.keys())[0]
    print(f"Using cover image: '{cover_image_file}")

    secret_message = input("Enter thee secret message you want to hide: ")
    output_stego_image_file = input("Enter the desired file name for stego image (e.g stego_image.png): ")

    hide_text(cover_image_file, secret_message, output_stego_image_file)
    print(f"Hiding process completed. Check '{output_stego_image_file}' in your files")


  elif choice == '2':
    print("\n--- Extract Text ---")
    print("Upload the stego image from which to extract text: ")
    uploaded = files.upload()

    if not uploaded:
      print("No file uploaded, Returning to the main menu. ")
      continue

    stego_image_file = list(uploaded.keys())[0]
    print(f"Using stego image: {stego_image_file}")

    extracted_message = extract_text(stego_image_file)
    if extracted_message:
      print(f"\nExtracted message: {extracted_message}")

    else:
      print("Could not extract message, check the image")



  elif choice == '3':
    print("Exiting Steganography Tool. ")
    break

  else:
    print("Invalid choice. Enter 1,2 or 3. ")



--- Steganography Tool ---
1. Hide text into an image
2. Extract text from an image
3. Exit


Enter your choice (1,2 or 3): 1

--- Hide Text ---
Please upload your cover image:


Saving cover_image.png to cover_image.png
Using cover image: 'cover_image.png
Enter thee secret message you want to hide: HI THERE, Its a secret message
Enter the desired file name for stego image (e.g stego_image.png): stego_image.png
Text hidden successfully in 'stego_image.png' within Colab's session
Hiding process completed. Check 'stego_image.png' in your files


--- Steganography Tool ---
1. Hide text into an image
2. Extract text from an image
3. Exit


Enter your choice (1,2 or 3): 2

--- Extract Text ---
Upload the stego image from which to extract text: 


Saving stego_image.png to stego_image (1).png
Using stego image: stego_image (1).png

Extracted message: HI THERE, Its a secret message


--- Steganography Tool ---
1. Hide text into an image
2. Extract text from an image
3. Exit


Enter your choice (1,2 or 3): 3
Exiting Steganography Tool. 
